<a href="https://colab.research.google.com/github/devsiddhant/Colab_operations/blob/master/YT_DLP_with_rclone_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 📦 Download and install rclone binary automatically
import os
import zipfile
import shutil

# Download latest rclone release for Linux amd64
url = "https://downloads.rclone.org/rclone-current-linux-amd64.zip"
zip_path = "/content/rclone.zip"
extract_dir = "/content/rclone_extract"

# Download and unzip
print("⬇️ Downloading rclone...")
!curl -L -o "{zip_path}" "{url}"

print("📦 Extracting...")
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Move the rclone binary to ~/.local/bin
bin_path = os.path.expanduser("~/.local/bin")
os.makedirs(bin_path, exist_ok=True)

# Find the extracted rclone binary path
extracted_dir = next((d for d in os.listdir(extract_dir) if d.startswith("rclone-")), None)
rclone_binary_src = os.path.join(extract_dir, extracted_dir, "rclone")
rclone_binary_dest = os.path.join(bin_path, "rclone")

# Copy and set executable
shutil.copy2(rclone_binary_src, rclone_binary_dest)
os.chmod(rclone_binary_dest, 0o755)

# Update PATH for current session
%env PATH=/root/.local/bin:$PATH

# Confirm installation
print("✅ Rclone installed successfully at ~/.local/bin/rclone\n")
!rclone version


In [ ]:
#@title Upload your rclone.conf file
from google.colab import files
import os

print("📁 Upload your rclone.conf (found locally at ~/.config/rclone/rclone.conf)")
uploaded = files.upload()

config_dir = "/root/.config/rclone"
os.makedirs(config_dir, exist_ok=True)

for name in uploaded:
    if name.endswith(".conf"):
        os.rename(name, os.path.join(config_dir, "rclone.conf"))
        print(f"✔️ Uploaded and moved {name} to {config_dir}/rclone.conf")


In [ ]:
#@title Install yt-dlp and required dependencies
!pip install -U yt-dlp
!apt-get install -y ffmpeg


In [ ]:
#@title Prompt to choose rclone remote for upload
import subprocess

# List available rclone remotes
remotes = subprocess.check_output(['rclone', 'listremotes']).decode('utf-8').splitlines()

# Ask user for their choice of remote
remote = input(f"Choose your rclone remote from the following: {', '.join(remotes)}\n")

# Validate the input
if remote not in remotes:
    raise ValueError("Invalid remote selected.")


In [ ]:
#@title Config for videos
import yt_dlp

def download_video(url, remote):
    # yt-dlp options to download the best quality video up to 1080p and audio, merge them
    ydl_opts = {
        'format': 'bestaudio/best[height<=1080]',
        'outtmpl': '/tmp/%(title)s.%(ext)s',  # Temporary download location
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferredformat': 'mp4',  # Convert to mp4 format
        }],
        'merge_output_format': 'mp4',  # Ensure audio and video are merged as mp4
        'noplaylist': True,  # If a playlist is given, treat as individual videos
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        file_path = f"/tmp/{info['title']}.mp4"
        print(f"Downloaded: {file_path}")

        # Upload to rclone remote
        subprocess.run(['rclone', 'copy', file_path, f'{remote}:/path/to/remote/'])
        print(f"Uploaded {file_path} to {remote}")


In [ ]:
#@title Config for playlists
def download_playlist(url, remote):
    # yt-dlp options to download the best quality video up to 1080p and audio, merge them
    ydl_opts = {
        'format': 'bestaudio/best[height<=1080]',
        'outtmpl': '/tmp/%(title)s.%(ext)s',  # Temporary download location
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferredformat': 'mp4',  # Convert to mp4 format
        }],
        'merge_output_format': 'mp4',  # Ensure audio and video are merged as mp4
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        # Iterate through each video in the playlist and upload
        for entry in info['entries']:
            file_path = f"/tmp/{entry['title']}.mp4"
            print(f"Downloaded: {file_path}")

            # Upload to rclone remote
            subprocess.run(['rclone', 'copy', file_path, f'{remote}:/path/to/remote/'])
            print(f"Uploaded {file_path} to {remote}")


In [ ]:
#@title Enter video link and Download
video_url = input("Enter the video URL: ")
download_video(video_url, remote)


In [ ]:
#@title Enter video link and Download
video_url = input("Enter the video URL: ")
download_video(video_url, remote)
